# Analyse processed Tweets

Problems:

- Commas within text make it difficult to read files > when preprocessing, add a special delimiter to mark the beginning and the start of the Tweet content

In [1]:
# Useful starting lines
%matplotlib inline
import copy
import numpy as np 
#import matplotlib as mpl
#import datetime as dt
import matplotlib.pyplot as plt
import pandas as pd
import re
import glob
import ast
from datetime import datetime
from collections import Counter
import itertools
import nltk
nltk.download("stopwords")
from nltk import bigrams 
from nltk.corpus import stopwords
import string
import locale
import reverse_geocoder as rg
import multiprocessing as mp
import dask.dataframe as dd
locale.setlocale(locale.LC_TIME, "en_US.utf8")
%load_ext autoreload

[nltk_data] Downloading package stopwords to /home/servan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# 1. Load and combine Data

In [2]:
#idea read whole line >  then parse
def load_csv_data_old(data_path, sub_sample=True):
    """Loads data and transforms it into a pandas dataframe"""
    
    #extracts tweet ID and timestamp
    tw1 = pd.read_csv(data_path,sep=",",usecols=[0,1],encoding = "ISO-8859-1",header=None)
    
    #extracts tweet content
    #tw2 = pd.read_csv(data_path,sep="[0-9],|,\"\[",usecols=[2],encoding = "ISO-8859-1",header=None,engine="python")
    
    #extract coordinates & location tag
    #tw3 = pd.read_csv(data_path,sep=",\"\[|\]\",",usecols=[1,2],encoding = "ISO-8859-1",header=None,engine="python")
    
    #merge everything
    #tw = pd.concat((tw1,tw2,tw3),axis=1)
    
    #extract complete string per row
    tw = pd.read_csv(data_path,sep="NotADelimiter",usecols=[0,1],encoding = "ISO-8859-1",header=None,engine="python")  
    
    #extract tweet ID and timestamp
    s = tw[0].apply(lambda x: x.split(","))
    tw["id"] = s.apply(lambda x: x[0])
    tw["time"] = s.apply(lambda x: x[1] if len(x) > 1 else None)
    
    #extract tweet content
    s = tw[0].apply(lambda x: re.split("[0-9],|,\"\[",x))
    tw["text"] = s.apply(lambda x: x[2] if len(x) > 2 else None)
    
    #extract coordinates & location tag
    s = tw[0].apply(lambda x: re.split(",\"\[|\]\",",x))
    tw["coord"] = s.apply(lambda x: x[1] if len(x)> 1 else None)
    tw["loc_tag"] = s.apply(lambda x: x[2] if len(x) > 2 else None)
    #tw3 = pd.read_csv(data_path,sep=",\"\[|\]\",",usecols=[1,2],encoding = "ISO-8859-1",header=None,engine="python")
    
    
    # sub-sample
    if sub_sample:
        tw = tw[::50]
    return tw

def load_csv_data(data_path, sub_sample=True):
    """Loads data and transforms it into a pandas dataframe"""
    
    #extracts tweet ID and timestamp
    tw = pd.read_csv(data_path,encoding = "ISO-8859-1",header=None,engine="python")
    print(data_path)
    # sub-sample
    if sub_sample:
        tw = tw[::50]
    return tw

"""Read all files in Data directory, load & parse them and concatenate resulting DataFrames"""
#files = glob.glob('./Data/*.csv')
files = glob.glob('/mount/SDF/Dump2/*.csv')
tw = pd.DataFrame
first = True
for data_path in files:
    tw_temp = load_csv_data(data_path=data_path, sub_sample=True)
    if first:
        tw = tw_temp
        first = False
    else:
        tw = pd.concat((tw,tw_temp),axis=0,ignore_index=False)
tw.index = range(0,np.shape(tw)[0])
tw.columns = ["id","time","text","coord","loc_tag"]

In [4]:
files = glob.glob('/mount/SDF/Dump2/*.csv')
files = files[0:20]

if __name__ == '__main__':
    p = mp.Pool(mp.cpu_count()-2)
    pandas_list = p.map(load_csv_data, files)
    print("End")

tw = pd.concat(pandas_list,axis=0,ignore_index=False)
tw.index = range(0,np.shape(tw)[0])
tw.columns = ["id","time","text","coord","loc_tag"]

/mount/SDF/Dump2/tweets2015042910.csv
/mount/SDF/Dump2/tweets2016012910.csv
/mount/SDF/Dump2/tweets2016091512.csv
/mount/SDF/Dump2/tweets2015123113.csv
/mount/SDF/Dump2/tweets2015081208.csv
/mount/SDF/Dump2/tweets2016092707.csv
/mount/SDF/Dump2/tweets2015062815.csv
/mount/SDF/Dump2/tweets2015031006.csv
/mount/SDF/Dump2/tweets2016082816.csv
/mount/SDF/Dump2/tweets2015050316.csv
/mount/SDF/Dump2/tweets2015082006.csv
/mount/SDF/Dump2/tweets2016050221.csv
/mount/SDF/Dump2/tweets2016082802.csv
/mount/SDF/Dump2/tweets2015080121.csv
/mount/SDF/Dump2/tweets2015060819.csv
/mount/SDF/Dump2/tweets2016061401.csv
/mount/SDF/Dump2/tweets2016061519.csv
/mount/SDF/Dump2/tweets2016081704.csv
/mount/SDF/Dump2/tweets2016070119.csv
/mount/SDF/Dump2/tweets2015121602.csv
End


In [5]:
tw
tw.to_pickle("PickledTweets.pkl")

In [12]:
tw = pd.read_pickle("PickledTweets.pkl")
tw

,id,time,text,coord,loc_tag
0,776376269281849344,Thu Sep 15 11:05:01 +0000 2016,@julietchiapell0 thanks Juliet!!,"[[[-77.479597, 38.802143], [-77.479597, 38.880...",place
1,776376290500894720,Thu Sep 15 11:05:06 +0000 2016,I can't choose a side. Man on The Moon make me...,"[[[-77.037081, 38.688359], [-77.037081, 38.817...",place
2,776376318971809792,Thu Sep 15 11:05:13 +0000 2016,"@Tavi59 saw him in Denver last year, Taj Mahal...","[[[-105.148074, 40.47168], [-105.148074, 40.65...",place
3,776376343990763520,Thu Sep 15 11:05:19 +0000 2016,damn never seen a backflip on a dick https://t...,"[[[-75.280284, 39.871811], [-75.280284, 40.137...",place
4,776376363460730880,Thu Sep 15 11:05:24 +0000 2016,They time when the @Starbucks baristas not on...,"[[[-76.323459, 40.138596], [-76.323459, 40.166...",place
5,776376386017718272,Thu Sep 15 11:05:29 +0000 2016,@ConnValleyLax @ZimaGear nice work - as always...,"[[[-77.753839, 43.145239], [-77.753839, 43.180...",place
6,776376405907079169,Thu Sep 15 11:05:34 +0000 2016,You stole my avi @MahoneyKelli,"[[[-91.51308, 36.970298], [-91.51308, 42.50830...",place
7,776376426245320704,Thu Sep 15 11:05:39 +0000 2016,My pillow smells like @wild_hvnt95 and it's great,"[[[-83.127063, 39.791505], [-83.127063, 39.907...",place
8,776376442615504896,Thu Sep 15 11:05:42 +0000 2016,already hit the snooze button 3 times this mor...,"[[[-87.634643, 24.396308], [-87.634643, 31.001...",place
9,776376455781453833,Thu Sep 15 11:05:46 +0000 2016,"If you're looking for work in #Orlando, FL, ch...","[-81.3792365, 28.5383355]",coordinates


map
oder
mapasync
pool.map
multiprocessing 
pool
mapaysnc

https://github.com/epfml/sent2vec/blob/master/get_sentence_embeddings_from_pre-trained_models.ipynb

gridsearch.cv > scikitlearn

In [13]:
#transform string coordinates into floats
dtw_coord = dd.from_pandas(tw["coord"], npartitions=mp.cpu_count()-1)
dtw_coord = dtw_coord.map(lambda x: re.findall(r"[-+]?\d*\.\d+|\d+",x) if isinstance(x,str) else None)
tw["coord"] = dtw_coord.map(lambda x: [float(i) for i in x] if isinstance(x,list) else None).compute()

In [14]:
#tw["coord"] = tw["coord"].apply(lambda x: re.findall(r"[-+]?\d*\.\d+|\d+",x) if isinstance(x,str) else None)
#tw["coord"] = tw["coord"].apply(lambda x: [float(i) for i in x] if isinstance(x,list) else None)

In [15]:
"""assign a single coordinate to rows with location tag "place"""
indx = tw["loc_tag"]=="place"
tw_place = dd.from_pandas(tw[indx]["coord"],npartitions=mp.cpu_count()-1)
#tw_place = tw_place.apply(lambda x: x.replace("\"",""))
#tw_place =tw_place.apply(lambda x: ast.literal_eval(x))

#calculate the mean of the coordinates given; 
#because the coordinates are very close to each other, we can treat the surface as flat
def calc_mean(x):
    if x == None:
        return(None)
    else:
        lons = [x[i] for i in range(0,len(x),2)]
        lats = [x[i] for i in range(1,len(x),2)]
    lon = np.mean(lons)
    lat = np.mean(lats)
    return([lon,lat])

tw_place = tw_place.map(calc_mean,meta=tw_place)
tw.loc[indx,"coord"] = tw_place.compute()

In [16]:
#convert time strings to datetime formate
#tw["time"] = tw["time"].apply(lambda x: re.sub("\+0000\s","",str(x)) if re.search("\+0000\s",str(x)) else None)
#tw["time"] = tw["time"].apply(lambda x: datetime.strptime(x,"%a %b %d %H:%M:%S %Y") if isinstance(x,str) else None)
dtw_time = dd.from_pandas(tw["time"], npartitions=mp.cpu_count()-1)
dtw_time = dtw_time.map(lambda x: re.sub("\+0000\s","",str(x)) if re.search("\+0000\s",str(x)) else None,meta=dtw_time)
tw["time"] = dtw_time.map(lambda x: datetime.strptime(x,"%a %b %d %H:%M:%S %Y") if isinstance(x,str) else None,meta=dtw_time).compute()

In [17]:
#convert id-string to int
#tw["id"] = tw["id"].apply(lambda x: float(x) if re.search("^[0-9]{18}$",x) else None)

In [18]:
#remove first colum with full row
#del tw[0]

def find_nones(df):
    indices = df.apply(lambda x: False if x.isnull().any() else True,axis=1)
    return(indices)


#remove all rows which contain None in any row
dtw_missing = dd.from_pandas(tw,npartitions=mp.cpu_count()-1)
indx = dtw_missing.map_partitions(find_nones,meta=(None,"bool"))
tw = tw[indx.compute()]
print("Removed a total of "+ str(sum(~indx.compute())) + " rows")

Removed a total of 190 rows


In [19]:
tw

,id,time,text,coord,loc_tag
0,776376269281849344,2016-09-15 11:05:01,@julietchiapell0 thanks Juliet!!,"[-77.438513, 38.841163]",place
1,776376290500894720,2016-09-15 11:05:06,I can't choose a side. Man on The Moon make me...,"[-76.9924405, 38.752715]",place
2,776376318971809792,2016-09-15 11:05:13,"@Tavi59 saw him in Denver last year, Taj Mahal...","[-105.0639425, 40.5641905]",place
3,776376343990763520,2016-09-15 11:05:19,damn never seen a backflip on a dick https://t...,"[-75.117998, 40.0048655]",place
4,776376363460730880,2016-09-15 11:05:24,They time when the @Starbucks baristas not on...,"[-76.303738, 40.15251]",place
5,776376386017718272,2016-09-15 11:05:29,@ConnValleyLax @ZimaGear nice work - as always...,"[-77.712567, 43.1626595]",place
6,776376405907079169,2016-09-15 11:05:34,You stole my avi @MahoneyKelli,"[-89.2665075, 39.7393005]",place
7,776376426245320704,2016-09-15 11:05:39,My pillow smells like @wild_hvnt95 and it's great,"[-83.0677285, 39.849473]",place
8,776376442615504896,2016-09-15 11:05:42,already hit the snooze button 3 times this mor...,"[-83.804475, 27.698682]",place
9,776376455781453833,2016-09-15 11:05:46,"If you're looking for work in #Orlando, FL, ch...","[-81.3792365, 28.5383355]",coordinates


In [20]:
#Tokenize & Count tokens in tweet text
#from https://marcobonzanini.com/2015/03/09/mining-twitter-data-with-python-part-2/
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)

def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False,stopwords_rm=False,**kwargs):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    if stopwords_rm:
        tokens = [token for token in tokens if token not in kwargs["stop_words"]]
    return tokens

In [29]:
dtw_tokens= dd.from_pandas(tw["text"], npartitions=mp.cpu_count()-1)
dtw_tokens = dtw_tokens.map(preprocess,True,meta=dtw_tokens)
#tw["tokens"] = tw["text"].apply(preprocess,lowercase=True)
tw["tokens"] = dtw_tokens.compute()

/home/servan/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [30]:
#Count occurrence of each word
counts = Counter(itertools.chain.from_iterable(tw["tokens"]))

In [31]:
counts

Counter({'@WSUDLERSMITH': 2,
         '@rohitbhargava': 1,
         'lesbian': 2,
         'stresseful': 1,
         'shameful': 1,
         'https://t.co/GiAzAtpoHN': 1,
         'bidder': 1,
         'merece': 1,
         'bishhhhhh': 1,
         'shed': 1,
         'https://t.co/xKKpM1a7Yx': 1,
         'Wvu': 1,
         '@kennycide': 1,
         'http://t.co/wscFXwUXec': 1,
         'cerca': 3,
         '@MelissaFox26': 1,
         'steadfastly': 1,
         'https://t.co/4tIZ02A4nZ': 1,
         'ligeiro': 1,
         'Tweet': 7,
         't2': 1,
         'Nux': 1,
         '#geometric': 1,
         'cokehead': 1,
         '#hrvb': 1,
         'caga': 2,
         'http://t.co/DFjLyZyw1k': 1,
         '@cami_croft': 1,
         'https://t.co/1lzE6qH3vr': 1,
         'Norfolk': 2,
         '@TheRealDyll': 1,
         '#birdgang': 1,
         'beaten': 3,
         'https://t.co/sjjRnD0gRN': 1,
         'osrs': 1,
         '@iGetFreeHugs': 1,
         '@vavonni159': 1,
         'Lex

In [35]:
#remove punctuation and stopwords
punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['rt', 'via']
#tw["tokens_stop"] = tw["text"].apply(preprocess,lowercase=True,
                              #stopwords_rm = True, stop_words=stop)

dtw_tokens= dd.from_pandas(tw["text"], npartitions=mp.cpu_count()-1)
dtw_tokens = dtw_tokens.map(preprocess,(True,True, stop), meta=dtw_tokens)
tw["tokens_stop"] = dtw_tokens.compute()

/home/servan/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [36]:
tw

,id,time,text,coord,loc_tag,tokens,tokens_stop
0,776376269281849344,2016-09-15 11:05:01,@julietchiapell0 thanks Juliet!!,"[-77.438513, 38.841163]",place,"[@julietchiapell0, thanks, Juliet, !, !]","[@julietchiapell0, thanks, Juliet, !, !]"
1,776376290500894720,2016-09-15 11:05:06,I can't choose a side. Man on The Moon make me...,"[-76.9924405, 38.752715]",place,"[I, can't, choose, a, side, ., Man, on, The, M...","[I, can't, choose, a, side, ., Man, on, The, M..."
2,776376318971809792,2016-09-15 11:05:13,"@Tavi59 saw him in Denver last year, Taj Mahal...","[-105.0639425, 40.5641905]",place,"[@Tavi59, saw, him, in, Denver, last, year, ,,...","[@Tavi59, saw, him, in, Denver, last, year, ,,..."
3,776376343990763520,2016-09-15 11:05:19,damn never seen a backflip on a dick https://t...,"[-75.117998, 40.0048655]",place,"[damn, never, seen, a, backflip, on, a, dick, ...","[damn, never, seen, a, backflip, on, a, dick, ..."
4,776376363460730880,2016-09-15 11:05:24,They time when the @Starbucks baristas not on...,"[-76.303738, 40.15251]",place,"[They, time, when, the, @Starbucks, baristas, ...","[They, time, when, the, @Starbucks, baristas, ..."
5,776376386017718272,2016-09-15 11:05:29,@ConnValleyLax @ZimaGear nice work - as always...,"[-77.712567, 43.1626595]",place,"[@ConnValleyLax, @ZimaGear, nice, work, -, as,...","[@ConnValleyLax, @ZimaGear, nice, work, -, as,..."
6,776376405907079169,2016-09-15 11:05:34,You stole my avi @MahoneyKelli,"[-89.2665075, 39.7393005]",place,"[You, stole, my, avi, @MahoneyKelli]","[You, stole, my, avi, @MahoneyKelli]"
7,776376426245320704,2016-09-15 11:05:39,My pillow smells like @wild_hvnt95 and it's great,"[-83.0677285, 39.849473]",place,"[My, pillow, smells, like, @wild_hvnt95, and, ...","[My, pillow, smells, like, @wild_hvnt95, and, ..."
8,776376442615504896,2016-09-15 11:05:42,already hit the snooze button 3 times this mor...,"[-83.804475, 27.698682]",place,"[already, hit, the, snooze, button, 3, times, ...","[already, hit, the, snooze, button, 3, times, ..."
9,776376455781453833,2016-09-15 11:05:46,"If you're looking for work in #Orlando, FL, ch...","[-81.3792365, 28.5383355]",coordinates,"[If, you're, looking, for, work, in, #Orlando,...","[If, you're, looking, for, work, in, #Orlando,..."


In [38]:
#create bigrams
#tw["bigrams"] = tw["tokens_stop"].apply(bigrams)
tw["bigrams"] = dd.from_pandas(tw["tokens_stop"],npartitions=mp.cpu_count()-1).map(bigrams).compute()

/home/servan/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [39]:
tw

,id,time,text,coord,loc_tag,tokens,tokens_stop,bigrams
0,776376269281849344,2016-09-15 11:05:01,@julietchiapell0 thanks Juliet!!,"[-77.438513, 38.841163]",place,"[@julietchiapell0, thanks, Juliet, !, !]","[@julietchiapell0, thanks, Juliet, !, !]",<generator object bigrams at 0x7f5c845b9af0>
1,776376290500894720,2016-09-15 11:05:06,I can't choose a side. Man on The Moon make me...,"[-76.9924405, 38.752715]",place,"[I, can't, choose, a, side, ., Man, on, The, M...","[I, can't, choose, a, side, ., Man, on, The, M...",<generator object bigrams at 0x7f5c845b9c50>
2,776376318971809792,2016-09-15 11:05:13,"@Tavi59 saw him in Denver last year, Taj Mahal...","[-105.0639425, 40.5641905]",place,"[@Tavi59, saw, him, in, Denver, last, year, ,,...","[@Tavi59, saw, him, in, Denver, last, year, ,,...",<generator object bigrams at 0x7f5c845b9a40>
3,776376343990763520,2016-09-15 11:05:19,damn never seen a backflip on a dick https://t...,"[-75.117998, 40.0048655]",place,"[damn, never, seen, a, backflip, on, a, dick, ...","[damn, never, seen, a, backflip, on, a, dick, ...",<generator object bigrams at 0x7f5c845b97d8>
4,776376363460730880,2016-09-15 11:05:24,They time when the @Starbucks baristas not on...,"[-76.303738, 40.15251]",place,"[They, time, when, the, @Starbucks, baristas, ...","[They, time, when, the, @Starbucks, baristas, ...",<generator object bigrams at 0x7f5c845b9938>
5,776376386017718272,2016-09-15 11:05:29,@ConnValleyLax @ZimaGear nice work - as always...,"[-77.712567, 43.1626595]",place,"[@ConnValleyLax, @ZimaGear, nice, work, -, as,...","[@ConnValleyLax, @ZimaGear, nice, work, -, as,...",<generator object bigrams at 0x7f5c845b9728>
6,776376405907079169,2016-09-15 11:05:34,You stole my avi @MahoneyKelli,"[-89.2665075, 39.7393005]",place,"[You, stole, my, avi, @MahoneyKelli]","[You, stole, my, avi, @MahoneyKelli]",<generator object bigrams at 0x7f5c845b9678>
7,776376426245320704,2016-09-15 11:05:39,My pillow smells like @wild_hvnt95 and it's great,"[-83.0677285, 39.849473]",place,"[My, pillow, smells, like, @wild_hvnt95, and, ...","[My, pillow, smells, like, @wild_hvnt95, and, ...",<generator object bigrams at 0x7f5c845b9570>
8,776376442615504896,2016-09-15 11:05:42,already hit the snooze button 3 times this mor...,"[-83.804475, 27.698682]",place,"[already, hit, the, snooze, button, 3, times, ...","[already, hit, the, snooze, button, 3, times, ...",<generator object bigrams at 0x7f5c845b9360>
9,776376455781453833,2016-09-15 11:05:46,"If you're looking for work in #Orlando, FL, ch...","[-81.3792365, 28.5383355]",coordinates,"[If, you're, looking, for, work, in, #Orlando,...","[If, you're, looking, for, work, in, #Orlando,...",<generator object bigrams at 0x7f5c845b9410>


In [40]:
#switch lat & lon position to work with reverse geocoder
#tw["coord"] = tw["coord"].apply(lambda x: [x[1],x[0]] if len(x)>1 else None)
tw["coord"] = dd.from_pandas(tw["coord"],npartitions=mp.cpu_count()-1).map(lambda x: [x[1],x[0]]).compute()

/home/servan/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [35]:
#tw = tw[[i is not None for i in tw["coord"]]]

In [7]:
#still getting too many open files error
#get state location for each tweet
def state_lookup(x):
    coords =[(i,j) for i,j in x.values.flatten()]
    loc_info = rg.search(coords)
    states = [i["admin1"] for i in loc_info]
    #state = loc_info[0]["admin1"]
    return(states)

def state_lookup(x):
    coords =[(i,j) for i,j in x.values.flatten()]
    loc_info = rg.search(coords)
    states = [i["admin1"] for i in loc_info]
    #state = loc_info[0]["admin1"]
    return(states)

#tw["state"] = state_lookup(tw["coord"])


In [8]:
dtw_coords = dd.from_pandas(tw["coord"],npartitions=mp.cpu_count()-1)
tw["state"] = dtw_coords.map_partitions(state_lookup,meta=dtw_coords).compute()

Loading formatted geocoded file...Loading formatted geocoded file...Loading formatted geocoded file...Loading formatted geocoded file...Loading formatted geocoded file...Loading formatted geocoded file...Loading formatted geocoded file...Loading formatted geocoded file...Loading formatted geocoded file...Loading formatted geocoded file...Loading formatted geocoded file...Loading formatted geocoded file...Loading formatted geocoded file...Loading formatted geocoded file...Loading formatted geocoded file...Loading formatted geocoded file...Loading formatted geocoded file...Loading formatted geocoded file...Loading formatted geocoded file...Loading formatted geocoded file...Loading formatted geocoded file...Loading formatted geocoded file...Loading formatted geocoded file...Loading formatted geocoded file...Loading formatted geocoded file...Loading formatted geocoded file...Loading formatted geocoded file...Loading formatted geocoded file...Loading formatted geocoded file...Loading format

OSError: [Errno 24] Too many open files

Traceback
---------
  File "/home/servan/anaconda3/lib/python3.5/site-packages/dask/async.py", line 291, in execute_task
    result = _execute_task(task, data)
  File "/home/servan/anaconda3/lib/python3.5/site-packages/dask/async.py", line 272, in _execute_task
    return func(*args2)
  File "/home/servan/anaconda3/lib/python3.5/site-packages/dask/dataframe/core.py", line 3148, in apply_and_enforce
    df = func(*args, **kwargs)
  File "<ipython-input-7-2aa7d2ea2ad6>", line 4, in state_lookup
    loc_info = rg.search(coords)
  File "/home/servan/anaconda3/lib/python3.5/site-packages/reverse_geocoder/__init__.py", line 293, in search
    return _rg.query(geo_coords)
  File "/home/servan/anaconda3/lib/python3.5/site-packages/reverse_geocoder/__init__.py", line 127, in query
    _, indices = self.tree.pquery(coordinates, k=1)
  File "/home/servan/anaconda3/lib/python3.5/site-packages/reverse_geocoder/cKDTree_MP.py", line 97, in pquery
  File "/home

In [3]:
#del tw["bigrams"]
tw.to_pickle("PickledTweets_states.pkl")

NameError: name 'tw' is not defined

In [4]:
tw = pd.read_pickle("PickledTweets_states.pkl")
tw

,id,time,text,coord,loc_tag,tokens,tokens_stop
0,776376269281849344,2016-09-15 11:05:01,@julietchiapell0 thanks Juliet!!,"[38.841163, -77.438513]",place,"[@julietchiapell0, thanks, Juliet, !, !]","[@julietchiapell0, thanks, Juliet, !, !]"
1,776376290500894720,2016-09-15 11:05:06,I can't choose a side. Man on The Moon make me...,"[38.752715, -76.9924405]",place,"[I, can't, choose, a, side, ., Man, on, The, M...","[I, can't, choose, a, side, ., Man, on, The, M..."
2,776376318971809792,2016-09-15 11:05:13,"@Tavi59 saw him in Denver last year, Taj Mahal...","[40.5641905, -105.0639425]",place,"[@Tavi59, saw, him, in, Denver, last, year, ,,...","[@Tavi59, saw, him, in, Denver, last, year, ,,..."
3,776376343990763520,2016-09-15 11:05:19,damn never seen a backflip on a dick https://t...,"[40.0048655, -75.117998]",place,"[damn, never, seen, a, backflip, on, a, dick, ...","[damn, never, seen, a, backflip, on, a, dick, ..."
4,776376363460730880,2016-09-15 11:05:24,They time when the @Starbucks baristas not on...,"[40.15251, -76.303738]",place,"[They, time, when, the, @Starbucks, baristas, ...","[They, time, when, the, @Starbucks, baristas, ..."
5,776376386017718272,2016-09-15 11:05:29,@ConnValleyLax @ZimaGear nice work - as always...,"[43.1626595, -77.712567]",place,"[@ConnValleyLax, @ZimaGear, nice, work, -, as,...","[@ConnValleyLax, @ZimaGear, nice, work, -, as,..."
6,776376405907079169,2016-09-15 11:05:34,You stole my avi @MahoneyKelli,"[39.7393005, -89.2665075]",place,"[You, stole, my, avi, @MahoneyKelli]","[You, stole, my, avi, @MahoneyKelli]"
7,776376426245320704,2016-09-15 11:05:39,My pillow smells like @wild_hvnt95 and it's great,"[39.849473, -83.0677285]",place,"[My, pillow, smells, like, @wild_hvnt95, and, ...","[My, pillow, smells, like, @wild_hvnt95, and, ..."
8,776376442615504896,2016-09-15 11:05:42,already hit the snooze button 3 times this mor...,"[27.698682, -83.804475]",place,"[already, hit, the, snooze, button, 3, times, ...","[already, hit, the, snooze, button, 3, times, ..."
9,776376455781453833,2016-09-15 11:05:46,"If you're looking for work in #Orlando, FL, ch...","[28.5383355, -81.3792365]",coordinates,"[If, you're, looking, for, work, in, #Orlando,...","[If, you're, looking, for, work, in, #Orlando,..."


In [51]:
tw["state"]

KeyError: 'state'

# 2. Build model

## 2.1 Naive Bayes

In [46]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()


In [40]:
from sklearn.datasets import fetch_20newsgroups
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

twenty_train = fetch_20newsgroups(subset='train',categories=categories, shuffle=True, random_state=42)

In [42]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [63]:
len(twenty_train.data)
tw <- tw[::50]

TypeError: bad operand type for unary -: 'Timestamp'

In [47]:
X_train_counts = count_vect.fit_transform(tw["text"])

In [62]:
X_train_counts.shape

<37621x57514 sparse matrix of type '<class 'numpy.int64'>'
	with 338646 stored elements in Compressed Sparse Row format>

In [51]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(37621, 57514)

In [52]:
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(37621, 57514)

In [55]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, tw["text"])

MemoryError: 